In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.utils import to_categorical

In [ ]:
NAME = 'DenseNet201_chu_7_c2_p0_bs32_data_0506_1110.23-0.76-0.60.hdf5'
LOAD_PATH='/work/power703/cgh/weight/'+NAME
OUT_PATH='/work/power703/cgh/weight/'+NAME+'.tflite'
TEST_PATH = '/work/power703/cgh/data_work/test0713/test/'
labels = [
#     "1",
    # "2",
    # "3",
    # "4",
    # "5",
#     "6",
    "7",
    # "8",
#     "9",
#     "10",
#     "11",
    # "12",
    # "13",
    # "14",
#     "15",
    "16"
]
classes = len(labels)

In [ ]:
model = keras.models.load_model(LOAD_PATH)
model.summary()

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [ ]:
# Save the TF Lite model.
with tf.io.gfile.GFile(OUT_PATH, 'wb') as f:
  f.write(tflite_model)

In [ ]:
def get_data_set(PATH):
    X = np.load(PATH + labels[0] + '.npy')
    y = np.zeros(X.shape[0])
    for i, label in enumerate(labels[1:]):
        x = np.load(PATH + label + '.npy')
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value=(i + 1)))
    assert X.shape[0] == len(y)
    return X, y

In [ ]:
x_test, y_test = get_data_set(TEST_PATH)
x_test = x_test.reshape(x_test.shape[0], 257, 500, 1)
y_test = to_categorical(y_test, num_classes=classes)

In [ ]:
tmp = np.zeros([1, 257, 500, 1])
ans = np.zeros([x_test.shape[0], classes])
tmp[0] = x_test[0]
input_data = np.array(tmp, dtype=np.float32)

In [ ]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=OUT_PATH)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
for i in range(len(x_test)):
    tmp[0] = x_test[i]
    input_data = np.array(tmp, dtype=np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    ans[i] = output_data
    print('.', end='')

In [ ]:
predict_label = np.argmax(ans, axis=1)
y_true = []
for i in range(len(y_test)):
    y_true.append(int(np.argmax(y_test[i])))  

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.3f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
plt.figure(figsize=[15, 10])
cnf_matrix = confusion_matrix(y_true, predict_label)
plot_confusion_matrix(cnf_matrix, classes=labels, normalize=True,
                      title=NAME)

plt.savefig(NAME+'.png', bbox_inches='tight')

print(classification_report(y_true, predict_label, target_names=labels, digits=3))
print('NAME: ', NAME)